In [1]:
from cosmikyu import gan, config, datasets, transforms, utils
from cosmikyu import nn as cnn
import os
import torch
import mlflow
from orphics import maps
import torchsummary


%matplotlib inline
%load_ext autoreload
%autoreload 2

In [16]:
data_dir = config.default_data_dir
sehgal_dir = os.path.join(data_dir, 'sehgal')
cuda = True
compts = ["kappa", "ksz", "tsz", "ir_pts", "rad_pts"]
compt_idxes = [0, 1, 2, 3, 4]
shape = (len(compt_idxes), 128, 128)
latent_dim = 256
sample_interval = 200
save_interval = 5
batch_size = 8
nepochs = 100
norm_info_file = "/home/dwhan89/workspace/cosmikyu/data/sehgal/201020_logzshrink_normalization_info_validation.npz"

In [17]:
_, wcs = maps.rect_geometry(width_arcmin=64., px_res_arcmin=0.5)

# Configure data loader
os.makedirs(data_dir, exist_ok=True)
os.makedirs(sehgal_dir, exist_ok=True)
SDN = transforms.SehgalDataNormalizerScaledLogZShrink(norm_info_file)
SC = transforms.SehgalSubcomponets(compt_idxes)
RF = transforms.RandomFlips(p_v=0.5, p_h=0.5)
SDS_train = datasets.SehgalDataSet(sehgal_dir, data_type="train141020", transforms=[SDN, RF, SC], dummy_label=True)

dataloader = torch.utils.data.DataLoader(
    SDS_train,
    batch_size=batch_size,
    shuffle=True,
)


In [18]:
STanh = cnn.ScaledTanh(30., 2./30.)
#experiment_id = "c0eb302516f64c87adf202830677a0da"
#model_dir = "/home/dwhan89/workspace/cosmikyu/output/sehgal_dcganwgp_211020/{}/model".format(experiment_id)
PIXGAN = gan.PIXGAN("sehgal_pixgan_251020", shape, nconv_fcgen=64,
                          nconv_fcdis=64, cuda=cuda, ngpu=4, nconv_layer_gen=4, nconv_layer_disc=4, kernal_size=5, stride=2,
                          padding=2, output_padding=1, gen_act=STanh, nin_channel=1, nout_channel=4, nthresh_layer_gen=3, nthresh_layer_disc=2)
#PIXGAN = gan.PIXGAN_WGP("sehgal_pixgan_251020", shape, nconv_fcgen=64,
#                          nconv_fcdis=64, cuda=cuda, ngpu=4, nconv_layer_gen=4, nconv_layer_disc=4, kernal_size=5, stride=2,
#                          padding=2, output_padding=1, gen_act=STanh, nin_channel=1, nout_channel=4, nthresh_layer_gen=3, nthresh_layer_disc=2)
#torchsummary.summary(PIXGAN.discriminator, (5,128,128), device="cpu")
#torchsummary.summary(PIXGAN.generator, (1,128,128), device="cpu")

/home/dwhan89/.miniconda3/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
Traceback (most recent call last):
  File "/home/dwhan89/.miniconda3/lib/python3.7/site-packages/mlflow/store/tracking/file_store.py", line 197, in list_experiments
    experiment = self._get_experiment(exp_id, view_type)
  File "/home/dwhan89/.miniconda3/lib/python3.7/site-packages/mlflow/store/tracking/file_store.py", line 260, in _get_experiment
    meta = read_yaml(experiment_dir, FileStore.META_DATA_FILE_NAME)
  File "/home/dwhan89/.miniconda3/lib/python3.7/site-packages/mlflow/utils/file_utils.py", line 167, in read_yaml
    raise MissingConfigException("Yaml file '%s' does not exist." % file_path)
mlflow.exceptions.M

In [1]:
mlflow.set_experiment(PIXGAN.identifier)
with mlflow.start_run(experiment_id=PIXGAN.experiment.experiment_id) as mlflow_run:
    torch.cuda.empty_cache()
    PIXGAN.train(
        dataloader,
        nepochs=nepochs,
        ncritics=1,
        sample_interval=sample_interval,
        save_interval=save_interval,
        load_states=True,
        save_states=True,
        verbose=True,
        mlflow_run=mlflow_run,
        lr=1e-4,
        betas=(0.5, 0.9),
        lambda_l1=100.
    )


NameError: name 'mlflow' is not defined

In [27]:
latent_dim = 128
DCGAN = gan.DCGAN_WGP("sehgal_dcgan_251020", shape,latent_dim, nconv_fcgen=64,
                          nconv_fcdis=64, cuda=cuda, ngpu=4, nconv_layer_gen=4, nconv_layer_disc=4, kernal_size=5, stride=2,
                          padding=2, output_padding=1, gen_act=STanh)


Traceback (most recent call last):
  File "/home/dwhan89/.miniconda3/lib/python3.7/site-packages/mlflow/store/tracking/file_store.py", line 197, in list_experiments
    experiment = self._get_experiment(exp_id, view_type)
  File "/home/dwhan89/.miniconda3/lib/python3.7/site-packages/mlflow/store/tracking/file_store.py", line 260, in _get_experiment
    meta = read_yaml(experiment_dir, FileStore.META_DATA_FILE_NAME)
  File "/home/dwhan89/.miniconda3/lib/python3.7/site-packages/mlflow/utils/file_utils.py", line 167, in read_yaml
    raise MissingConfigException("Yaml file '%s' does not exist." % file_path)
mlflow.exceptions.MissingConfigException: Yaml file '/home/dwhan89/workspace/cosmikyu/cosmikyu/../output/mlruns/mlruns/meta.yaml' does not exist.
Traceback (most recent call last):
  File "/home/dwhan89/.miniconda3/lib/python3.7/site-packages/mlflow/store/tracking/file_store.py", line 197, in list_experiments
    experiment = self._get_experiment(exp_id, view_type)
  File "/home/dwhan8

In [ ]:
mlflow.set_experiment(DCGAN.identifier)
with mlflow.start_run(experiment_id=DCGAN.experiment.experiment_id) as mlflow_run:
    torch.cuda.empty_cache()
    PIXGAN.train(
        dataloader,
        nepochs=nepochs,
        ncritics=5,
        sample_interval=sample_interval,
        save_interval=save_interval,
        load_states=True,
        save_states=True,
        verbose=True,
        mlflow_run=mlflow_run,
        lr=2e-4,
        betas=(0.5, 0.999),
        lambda_gp=10.
    )

Traceback (most recent call last):
  File "/home/dwhan89/.miniconda3/lib/python3.7/site-packages/mlflow/store/tracking/file_store.py", line 197, in list_experiments
    experiment = self._get_experiment(exp_id, view_type)
  File "/home/dwhan89/.miniconda3/lib/python3.7/site-packages/mlflow/store/tracking/file_store.py", line 260, in _get_experiment
    meta = read_yaml(experiment_dir, FileStore.META_DATA_FILE_NAME)
  File "/home/dwhan89/.miniconda3/lib/python3.7/site-packages/mlflow/utils/file_utils.py", line 167, in read_yaml
    raise MissingConfigException("Yaml file '%s' does not exist." % file_path)
mlflow.exceptions.MissingConfigException: Yaml file '/home/dwhan89/workspace/cosmikyu/cosmikyu/../output/mlruns/mlruns/meta.yaml' does not exist.


loading saved states 
failed to load saved states
saving states _0
> /home/dwhan89/workspace/cosmikyu/cosmikyu/gan.py(530)_eval_discriminator_loss()
    528 
    529         import pdb; pdb.set_trace()
--> 530         gradients = autograd.grad(
    531             outputs=disc_interp,
    532             inputs=interp_data,

ipdb> storage.shape
torch.Size([32, 1])
ipdb> n
> /home/dwhan89/workspace/cosmikyu/cosmikyu/gan.py(531)_eval_discriminator_loss()
    529         import pdb; pdb.set_trace()
    530         gradients = autograd.grad(
--> 531             outputs=disc_interp,
    532             inputs=interp_data,
    533             grad_outputs=storage,

ipdb> disc_interp.shape
torch.Size([32, 1, 1, 1])
ipdb> n
> /home/dwhan89/workspace/cosmikyu/cosmikyu/gan.py(532)_eval_discriminator_loss()
    530         gradients = autograd.grad(
    531             outputs=disc_interp,
--> 532             inputs=interp_data,
    533             grad_outputs=storage,
    534             create